<a href="https://colab.research.google.com/github/EdiNel0407/us-ie-big-data-technologies/blob/main/postblock2/q4/postblock2_q4_3_koalas_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PostBlock2 — Q4.3: Koalas-style API (pyspark.pandas)

In [2]:
!pip -q install pyspark==3.5.1 "numpy<2.0"

import os, numpy as np
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
print("NumPy:", np.__version__)



NumPy: 2.0.2


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Q4.3_Koalas_RF").getOrCreate()
print("Spark:", spark.version)



Spark: 3.5.1


In [8]:
import pyspark.pandas as ps
ps.set_option("compute.ops_on_diff_frames", True)  # <— allow mixing Series from different frames

# now this works:
noise = ps.Series(np.random.default_rng(23).random(len(kdf)),
                  index=kdf.index.to_pandas(), name="noise")

kdf["label"] = (
    (kdf.x1*0.8 + kdf.x2*0.6 - kdf.x3*0.7 + kdf.x4*0.2 + noise*0.3) > 0.8
).astype("int")





/usr/local/lib/python3.12/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas Index is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [9]:
# Convert to Spark DF and train RF
sdf = kdf.to_spark()

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

assembler = VectorAssembler(inputCols=["x1","x2","x3","x4"], outputCol="features")
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=8, seed=42)
pipeline = Pipeline(stages=[assembler, rf])

train_df, test_df = sdf.randomSplit([0.8, 0.2], seed=123)
model = pipeline.fit(train_df)
pred  = model.transform(test_df)

acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(pred)
print(f"Test Accuracy (Koalas prep): {acc:.4f}")

model.write().overwrite().save("/content/models/q4_3_koalas_rf_model")


/usr/local/lib/python3.12/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


Test Accuracy (Koalas prep): 0.9182
